# A Demonstration of the Diffie&ndash;Hellman Key Agreement Protocol
The Diffie&ndash;Hellman protocol illustrated below permits two parties (Alice and Bob) to exchange a confidential message on an insecure channel, and prevents a third party (Eve) who intercepts the message from learning its content.

Whitfield Diffie and Martin Hellman first introduced this protocol in their seminal 1976 paper [_New Directions in Cryptography_](https://ee.stanford.edu/~hellman/publications/24.pdf). This protocol solved a problem that had afflicted all previously known ciphers; that of exchanging secret encryption keys _securely_ between trusted parties. The so&ndash;called _Diffie&ndash;Hellman key agreement protocol_ remains to this day the de facto standard for negotiating shared keys over insecure channels.

In the same paper, Diffie and Hellman imagined a novel cryptosystem in which a publicly&ndash;known key could be used to encrypt a message that only the owner of a different, _secret_ key could decrypt. Such a system&mdash;what they termed a _public&ndash;key_ cryptosystem&mdash;could further be employed to guarantee the authenticity of a message (a property that encryption alone could not provide), via a method they called _digital signature_.

Although Diffie and Hellman failed to realize a practical implementation for such a public&ndash;key cryptosystem, they did succeed in laying the essential groundwork for it (it would fall to Ronald Rivest, Adi Shamir and Leonard Adlemen to flesh it out in their 1978 paper [_A Method for Obtaining Digital Signatures and Public&ndash;Key Cryptosystems_](https://people.csail.mit.edu/rivest/Rsapaper.pdf)).

However, Diffie and Hellman did leave us with a practical implementation for secure key&ndash;exchange. The method is based on a [one&ndash;way function](https://en.wikipedia.org/wiki/One-way_function) that pits the difficulty of solving the [discrete logarithm problem](https://en.wikipedia.org/wiki/Discrete_logarithm) (or DLP) in large&ndash;order [multiplicative integer groups](https://en.wikipedia.org/wiki/Multiplicative_group) against the ease of multiplication in such groups.

### A Caveat
Note that the procedure as demonstrated provides no guarantee as to the authenticity of either party; that is, that the participants Alice or Bob (or both) are who they claim to be (see [_man&ndash;in&ndash;the&ndash;middle_](https://en.wikipedia.org/wiki/Man-in-the-middle_attack) attack for elaboration). Nor does it guarantee that the message encrypted in the procedure cannot be decrypted by an adversary if either private key used in the procedure is leaked at a future date (see [_forward secrecy_](https://en.wikipedia.org/wiki/Forward_secrecy) for elaboration). Protections against either vulnerability are not precluded by this implementation, but they are omitted here to minimize conceptual clutter.

## The Procedure
The following procedure, which utilizes the services of this repository's [dh](https://github.com/dchampion/crypto/tree/master/src/core/dh.py) module, illustrates a session in which Alice encrypts and transmits a  message to Bob over an insecure channel, which Bob successfully decrypts upon receipt. Meanwhile, an adversary (Eve) intercepts the message but, because the message is encrypted, has no feasible way of learning its content.

It is assumed that Alice, Bob and Eve all possess their own copy of the [dh](https://github.com/dchampion/crypto/tree/master/src/core/dh.py) module; but that Alice and Bob keep the encryption keys generated by this module secret.

Alice and Bob start by importing the [dh](https://github.com/dchampion/crypto/tree/master/src/core/dh.py) module, and a module [sym](https://github.com/dchampion/crypto/tree/master/src/tests/core/sym.py) that implements a symmetric cipher they have agreed in advance will be used to encrypt and decrypt messages (the [sym](https://github.com/dchampion/crypto/tree/master/src/tests/core/sym.py) cipher used here is for illustrative purposes only, and in a real&ndash;world setting would be replaced with a cryptographically&ndash;strong symmetric cipher such as _3DES_ or _AES_).

In [1]:
import os
os.chdir("../src")

# Alice and Bob (and Eve?) import the dh and sym modules required for the procedure
from core import dh
from tests.core import sym

Alice begins the procedure by computing a keypair consisting of a public key and a private key. She stores her keypair in `key_a` (the suffix `_a` identifies this keypair as belonging to Alice).

In [2]:
# Alice computes a keypair
key_a = dh.make_key()

In order to compute a fresh keypair, several preliminary steps must take place. These steps&mdash;which are hidden behind the implementation of the `dh.make_key()` function&mdash;produce a set of _domain parameters_ on which both Alice's and Bob's keypairs will be based. These domain parameters consist of a prime modulus $p$, a prime number $q$, which is the order of the smallest subgroup of the full group modulo $p$, and an integer $g$, which is a generator of this subgroup.

Alice must transmit not only her public key to Bob, but also the domain parameters. Note that the domain parameters are public, in that they may be observed by Eve with no compormise to the security of the system.

In [3]:
# Alice extracts the domain parameters and the public key from her keypair and transmits them to Bob
parameters = key_a.public_parameters()
pub_key_a = key_a.public_key()

Alice transmits the domain parameters `parameters` and her public key `pub_key_a` to Bob.

With the domain parameters supplied to him by Alice, Bob computes a keypair of his own.

In [4]:
# Bob computes a keypair, supplying the domain parameters Alice transmitted to him, and transmits his public key to Alice
key_b = dh.make_key(parameters)
pub_key_b = key_b.public_key()

Bob transmits a copy of his public key `pub_key_b` to Alice.

With possession of Bob's public key, Alice can now compute the session key required to encrypt a message she will transmit to Bob.

In [5]:
# Alice computes a session key
ses_key_a = key_a.make_session_key(pub_key_b)

Bob employs the same procedure as Alice to compute the session key.

In [6]:
# Bob computes a session key
ses_key_b = key_b.make_session_key(pub_key_a)

Herein lies the essential property of DH: Alice's public key `pub_key_a` raised to the power of Bob's private key is equal to Bob's public key `pub_key_b` raised to the power Alice's private key, both modulo $p$, thus producing identical session keys (`ses_key_a` and `ses_key_b`).

Where does this leave Eve? Having only seen `pub_key_a` and `pub_key_b`, but neither Alice's nor Bob's private keys, Eve cannot determine the value of the exponents used to compute them (i.e., she cannot solve the discrete logarithm problem, at least not in a reasonable amount of time). This is the _one&ndash;way_ function that lies at the heart of DH.

As with all traditional ciphers, the key Alice uses to encrypt the message is the same one Bob uses to decrypt it. The difference here is that encryption is not predicated on a secure _exchange_ of keys; rather, the keys are derived independently by Alice and Bob, and never transmitted over the insecure channel. Because of this, it is more accurate to describe DH as a key _agreement_ protocol rather than a key _exchange_ protocol.

If the essential property of DH holds, both session keys (`ses_key_a` and `ses_key_b`) must be equivalent.

In [7]:
ses_key_a == ses_key_b

True

With all the prerequisites for message encryption now in place, Alice composes a message for Bob.

In [8]:
# Alice composes a message (m) for Bob
m = "Encrypt me!"

She then encrypts this message by calling the `encrypt()` function of the module `sym`, which implements a simple symmetric cipher. To this function Alice passes her session key `ses_key_a` and her message `m`. The function returns a ciphertext of the message `c`.

In [9]:
# Alice encrypts the message, and transmits its ciphertext (c) to Bob
c = sym.encrypt(ses_key_a, m)

Alice transmits the ciphertext `c` to Bob over the insecure channel.

If we inspect the ciphertext, we see that to Eve it will appear as an incomprehensible string of bytes.

In [10]:
c

b'|\xe1\xe8\xea!\xd2\xba7\x02\x04^\xba\xcd\xc8\x8d\xa0qK\xf5\x06\xadO\xa3D\xf4\x04\x80a\xb7J\xa0\x9c'

Bob, having received the ciphertext from Alice, decrypts it by calling the `decrypt()` function of the symmetric cipher. To this fuction Bob passes his session key `ses_key_b` and the ciphertext `c`. The result `m1` should be identical to the plaintext `m` of Alice's original message.

In [11]:
# Bob decrypts Alice's ciphertext (c)
m1 = sym.decrypt(ses_key_b, c)

Inspecting `m1`, we see that Bob has successfully recovered the plaintext of Alice's message.

In [12]:
m1

'Encrypt me!'

Finally, Eve, having observed the domain parameters $p$, $q$ and $g$, and Alice's and Bob's public keys (`pub_key_a` and `pub_key_b`), but _not_ their private keys, has no feasible way of deriving the session key (`ses_key_a` or `ses_key_b`). Without the session key, Eve cannot decrypt the message.